In [ ]:
%%time
#####run only once (the first time)
#!pip install --upgrade spacy #incompatible with (!pip install spacy-pytorch-transformers)
#!pip install torch===1.4.0 torchvision===0.5.0 -f https://download.pytorch.org/whl/torch_stable.html  #this is useful only on jupyter
#!pip uninstall -y spacy-pytorch-transformers #this fixes #un
!pip install --upgrade spacy
#!pip install spacy-transformers
!pip install thinc_gpu_ops # uncomemnt thia if you want the gpu to run on kaggle
#!pip install spacy-pytorch-transformers #it is deprecated https://github.com/explosion/spaCy/issues/4382
!python -m spacy download en_trf_bertbaseuncased_lg  # only this on kaggle. Do not try to load a dataset with bert inside it won't work
#on kaggle, uncomment only this! 
#####

import sys
import pandas as pd
from collections import defaultdict
import spacy
from spacy.matcher import PhraseMatcher
import math   
from spacy.util import minibatch , compounding 
import random
import torch
import numpy as np
from progressbar import ProgressBar, Bar, Percentage
from sklearn.metrics import roc_auc_score
from os import listdir
from os.path import isfile, join
from sklearn.utils import shuffle
from spacy_transformers.util import cyclic_triangular_rate
from collections import Counter
import re
from thinc.neural.optimizers import Adam
from thinc.neural import Model
import matplotlib.pyplot as plt
import os

#enable gpu

import thinc_gpu_ops
print(thinc_gpu_ops.AVAILABLE) 
is_using_gpu = spacy.prefer_gpu()
if is_using_gpu:
    torch.set_default_tensor_type("torch.cuda.FloatTensor")
    print("done")


#end of gpu part

Collecting spacy  
  Downloading spacy-2.2.4-cp36-cp36m-manylinux1_x86_64.whl (10.6 MB)  
     |████████████████████████████████| 10.6 MB 2.8 MB/s   
Requirement already satisfied, skipping upgrade: setuptools in /opt/conda/lib/python3.6/site-packages (from spacy) (46.1.3.post20200330)  
Requirement already satisfied, skipping upgrade: wasabi<1.1.0,>=0.4.0 in /opt/conda/lib/python3.6/site-packages (from spacy) (0.6.0)  
Requirement already satisfied, skipping upgrade: murmurhash<1.1.0,>=0.28.0 in /opt/conda/lib/python3.6/site-packages (from spacy) (1.0.2)  
Requirement already satisfied, skipping upgrade: requests<3.0.0,>=2.13.0 in /opt/conda/lib/python3.6/site-packages (from spacy) (2.22.0)  
Requirement already satisfied, skipping upgrade: numpy>=1.15.0 in /opt/conda/lib/python3.6/site-packages (from spacy) (1.18.2)  
Requirement already satisfied, skipping upgrade: plac<1.2.0,>=0.9.6 in /opt/conda/lib/python3.6/site-packages (from spacy) (0.9.6)  
Requirement already satisfied, skipping upgrade: cymem<2.1.0,>=2.0.2 in /opt/conda/lib/python3.6/site-packages (from spacy) (2.0.3)  
Collecting thinc==7.4.0  
  Downloading thinc-7.4.0-cp36-cp36m-manylinux1_x86_64.whl (2.2 MB)  
     |████████████████████████████████| 2.2 MB 23.3 MB/s   
Requirement already satisfied, skipping upgrade: tqdm<5.0.0,>=4.38.0 in /opt/conda/lib/python3.6/site-packages (from spacy) (4.42.0)  
Requirement already satisfied, skipping upgrade: srsly<1.1.0,>=1.0.2 in /opt/conda/lib/python3.6/site-packages (from spacy) (1.0.2)  
Requirement already satisfied, skipping upgrade: preshed<3.1.0,>=3.0.2 in /opt/conda/lib/python3.6/site-packages (from spacy) (3.0.2)  
Requirement already satisfied, skipping upgrade: catalogue<1.1.0,>=0.0.7 in /opt/conda/lib/python3.6/site-packages (from spacy) (1.0.0)  
Requirement already satisfied, skipping upgrade: blis<0.5.0,>=0.4.0 in /opt/conda/lib/python3.6/site-packages (from spacy) (0.4.1)  
Requirement already satisfied, skipping upgrade: certifi>=2017.4.17 in /opt/conda/lib/python3.6/site-packages (from requests<3.0.0,>=2.13.0->spacy) (2019.11.28)  
Requirement already satisfied, skipping upgrade: chardet<3.1.0,>=3.0.2 in /opt/conda/lib/python3.6/site-packages (from requests<3.0.0,>=2.13.0->spacy) (3.0.4)  
Requirement already satisfied, skipping upgrade: idna<2.9,>=2.5 in /opt/conda/lib/python3.6/site-packages (from requests<3.0.0,>=2.13.0->spacy) (2.8)  
Requirement already satisfied, skipping upgrade: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /opt/conda/lib/python3.6/site-packages (from requests<3.0.0,>=2.13.0->spacy) (1.24.3)  
Requirement already satisfied, skipping upgrade: importlib-metadata>=0.20; python_version < "3.8" in /opt/conda/lib/python3.6/site-packages (from catalogue<1.1.0,>=0.0.7->spacy) (1.5.0)  
Requirement already satisfied, skipping upgrade: zipp>=0.5 in /opt/conda/lib/python3.6/site-packages (from importlib-metadata>=0.20; python_version < "3.8"->catalogue<1.1.0,>=0.0.7->spacy) (2.2.0)  
ERROR: allennlp 0.9.0 has requirement spacy<2.2,>=2.1.0, but you'll have spacy 2.2.4 which is incompatible.  
Installing collected packages: thinc, spacy  
  Attempting uninstall: thinc  
    Found existing installation: thinc 7.3.1  
    Uninstalling thinc-7.3.1:  
      Successfully uninstalled thinc-7.3.1  
  Attempting uninstall: spacy  
    Found existing installation: spacy 2.2.3  
    Uninstalling spacy-2.2.3:  
      Successfully uninstalled spacy-2.2.3  
Successfully installed spacy-2.2.4 thinc-7.4.0  
Collecting thinc_gpu_ops  
  Downloading thinc_gpu_ops-0.0.4.tar.gz (483 kB)  
     |████████████████████████████████| 483 kB 2.8 MB/s   
Requirement already satisfied: numpy>=1.7.0 in /opt/conda/lib/python3.6/site-packages (from thinc_gpu_ops) (1.18.2)  
Building wheels for collected packages: thinc-gpu-ops  
  Building wheel for thinc-gpu-ops (setup.py) ... done  
  Created wheel for thinc-gpu-ops: filename=thinc_gpu_ops-0.0.4-cp36-cp36m-linux_x86_64.whl size=216538 sha256=d7b0f751751b16303106d6270cddb36061c5f8a248f886dca8deebb548aaf389  
  Stored in directory: /root/.cache/pip/wheels/5b/21/70/2dabd331ebc5d25304e18c6bdbe621b16aebb1a61567733e6e  
Successfully built thinc-gpu-ops  
Installing collected packages: thinc-gpu-ops  
Successfully installed thinc-gpu-ops-0.0.4  
Collecting en_trf_bertbaseuncased_lg==2.2.0  
  Downloading https://github.com/explosion/spacy-models/releases/download/en_trf_bertbaseuncased_lg-2.2.0/en_trf_bertbaseuncased_lg-2.2.0.tar.gz (405.8 MB)  
     |████████████████████████████████| 405.8 MB 3.0 MB/s   
Requirement already satisfied: spacy>=2.2.1 in /opt/conda/lib/python3.6/site-packages (from en_trf_bertbaseuncased_lg==2.2.0) (2.2.4)  
Collecting spacy-transformers>=0.5.0  
  Downloading spacy-transformers-0.5.1.tar.gz (59 kB)  
     |████████████████████████████████| 59 kB 2.4 MB/s 
Requirement already satisfied: thinc==7.4.0 in /opt/conda/lib/python3.6/site-packages (from spacy>=2.2.1->en_trf_bertbaseuncased_lg==2.2.0) (7.4.0)  
Requirement already satisfied: blis<0.5.0,>=0.4.0 in /opt/conda/lib/python3.6/site-packages (from spacy>=2.2.1->en_trf_bertbaseuncased_lg==2.2.0) (0.4.1)   
Requirement already satisfied: preshed<3.1.0,>=3.0.2 in /opt/conda/lib/python3.6/site-packages (from spacy>=2.2.1->en_trf_bertbaseuncased_lg==2.2.0) (3.0.2 )  
Requirement already satisfied: setuptools in /opt/conda/lib/python3.6/site-packages (from spacy>=2.2.1->en_trf_bertbaseuncased_lg==2.2.0) (46.1.3.post20200330)  
Requirement already satisfied: requests<3.0.0,>=2.13.0 in /opt/conda/lib/python3.6/site-packages (from spacy>=2.2.1->en_trf_bertbaseuncased_lg==2.2.0) (2.22.0)  
Requirement already satisfied: wasabi<1.1.0,>=0.4.0 in /opt/conda/lib/python3.6/site-packages (from spacy>=2.2.1->en_trf_bertbaseuncased_lg==2.2.0) (0.6.0)  
Requirement already satisfied: tqdm<5.0.0,>=4.38.0 in /opt/conda/lib/python3.6/site-packages (from spacy>=2.2.1->en_trf_bertbaseuncased_lg==2.2.0) (4.42.0)  
Requirement already satisfied: cymem<2.1.0,>=2.0.2 in /opt/conda/lib/python3.6/site-packages (from spacy>=2.2.1->en_trf_bertbaseuncased_lg==2.2.0) (2.0.3)  
Requirement already satisfied: numpy>=1.15.0 in /opt/conda/lib/python3.6/site-packages (from spacy>=2.2.1->en_trf_bertbaseuncased_lg==2.2.0) (1.18.2)  
Requirement already satisfied: murmurhash<1.1.0,>=0.28.0 in /opt/conda/lib/python3.6/site-packages (from spacy>=2.2.1->en_trf_bertbaseuncased_lg==2.2.0) (1.0.2)  
Requirement already satisfied: catalogue<1.1.0,>=0.0.7 in /opt/conda/lib/python3.6/site-packages (from spacy>=2.2.1->en_trf_bertbaseuncased_lg==2.2.0) (1.0.0)  
Requirement already satisfied: plac<1.2.0,>=0.9.6 in /opt/conda/lib/python3.6/site-packages (from spacy>=2.2.1->en_trf_bertbaseuncased_lg==2.2.0) (0.9.6)  
Requirement already satisfied: srsly<1.1.0,>=1.0.2 in /opt/conda/lib/python3.6/site-packages (from spacy>=2.2.1->en_trf_bertbaseuncased_lg==2.2.0) (1.0.2)  
Collecting transformers<2.1.0,>=2.0.0  
  Downloading transformers-2.0.0-py3-none-any.whl (290 kB)  
     |████████████████████████████████| 290 kB 7.2 MB/s     
Requirement already satisfied: torch>=1.0.0 in /opt/conda/lib/python3.6/site-packages (from spacy-transformers>=0.5.0->en_trf_bertbaseuncased_lg==2.2.0) (1.4.0)  
Collecting torchcontrib<0.1.0,>=0.0.2  
  Downloading torchcontrib-0.0.2.tar.gz (11 kB)  
Requirement already satisfied: ftfy<6.0.0,>=5.0.0 in /opt/conda/lib/python3.6/site-packages (from spacy-transformers>=0.5.0->en_trf_bertbaseuncased_lg==2.2.0) (5.7)  
Collecting dataclasses<0.7,>=0.6  
  Downloading dataclasses-0.6-py3-none-any.whl (14 kB)  
Requirement already satisfied: importlib_metadata>=0.20 in /opt/conda/lib/python3.6/site-packages (from spacy-transformers>=0.5.0->en_trf_bertbaseuncased_lg==2.2.0) (1.5.0)  
Requirement already satisfied: idna<2.9,>=2.5 in /opt/conda/lib/python3.6/site-packages (from requests<3.0.0,>=2.13.0->spacy>=2.2.1->en_trf_bertbaseuncased_lg==2.2.0) (2.8)  
Requirement already satisfied: certifi>=2017.4.17 in /opt/conda/lib/python3.6/site-packages (from requests<3.0.0,>=2.13.0->spacy>=2.2.1->en_trf_bertbaseuncased_lg==2.2.0) (2019.11.28)  
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /opt/conda/lib/python3.6/site-packages (from requests<3.0.0,>=2.13.0->spacy>=2.2.1->en_trf_bertbaseuncased_lg==2.2.0) (1.24.3)  
Requirement already satisfied: chardet<3.1.0,>=3.0.2 in /opt/conda/lib/python3.6/site-packages (from requests<3.0.0,>=2.13.0->spacy>=2.2.1->en_trf_bertbaseuncased_lg==2.2.0) (3.0.4)  
Requirement already satisfied: regex in /opt/conda/lib/python3.6/site-packages (from transformers<2.1.0,>=2.0.0->spacy-transformers>=0.5.0->en_trf_bertbaseuncased_lg==2.2.0) (2020.2.20)  
Requirement already satisfied: sacremoses in /opt/conda/lib/python3.6/site-packages (from transformers<2.1.0,>=2.0.0->spacy-transformers>=0.5.0->en_trf_bertbaseuncased_lg==2.2.0) (0.0.38)  
Requirement already satisfied: boto3 in /opt/conda/lib/python3.6/site-packages (from transformers<2.1.0,>=2.0.0->spacy-transformers>=0.5.0->en_trf_bertbaseuncased_lg==2.2.0) (1.12.32)  
Requirement already satisfied: sentencepiece in /opt/conda/lib/python3.6/site-packages (from transformers<2.1.0,>=2.0.0->spacy-transformers>=0.5.0->en_trf_bertbaseuncased_lg==2.2.0) (0.1.85)  
Requirement already satisfied: wcwidth in /opt/conda/lib/python3.6/site-packages (from ftfy<6.0.0,>=5.0.0->spacy-transformers>=0.5.0->en_trf_bertbaseuncased_lg==2.2.0) (0.1.9)  
Requirement already satisfied: zipp>=0.5 in /opt/conda/lib/python3.6/site-packages (from importlib_metadata>=0.20->spacy-transformers>=0.5.0->en_trf_bertbaseuncased_lg==2.2.0) (2.2.0)  
Requirement already satisfied: joblib in /opt/conda/lib/python3.6/site-packages (from sacremoses->transformers<2.1.0,>=2.0.0->spacy-transformers>=0.5.0->en_trf_bertbaseuncased_lg==2.2.0) (0.14.1)  
Requirement already satisfied: six in /opt/conda/lib/python3.6/site-packages (from sacremoses->transformers<2.1.0,>=2.0.0->spacy-transformers>=0.5.0->en_trf_bertbaseuncased_lg==2.2.0) (1.14.0)   
Requirement already satisfied: click in /opt/conda/lib/python3.6/site-packages (from sacremoses->transformers<2.1.0,>=2.0.0->spacy-transformers>=0.5.0->en_trf_bertbaseuncased_lg==2.2.0) (7.1.1)  
Requirement already satisfied: s3transfer<0.4.0,>=0.3.0 in /opt/conda/lib/python3.6/site-packages (from boto3->transformers<2.1.0,>=2.0.0->spacy-transformers>=0.5.0->en_trf_bertbaseuncased_lg==2.2.0)   (0.3.3)
Requirement already satisfied: jmespath<1.0.0,>=0.7.1 in /opt/conda/lib/python3.6/site-packages (from boto3->transformers<2.1.0,>=2.0.0->spacy-transformers>=0.5.0->en_trf_bertbaseuncased_lg==2.2.0) (0.9.5)  
Requirement already satisfied: botocore<1.16.0,>=1.15.32 in /opt/conda/lib/python3.6/site-packages (from boto3->transformers<2.1.0,>=2.0.0->spacy-transformers>=0.5.0->en_trf_bertbaseuncased_lg==2.2.0) (1.15.32)  
Requirement already satisfied: docutils<0.16,>=0.10 in /opt/conda/lib/python3.6/site-packages (from botocore<1.16.0,>=1.15.32->boto3->transformers<2.1.0,>=2.0.0->spacy-transformers>=0.5.0-  >en_trf_bertbaseuncased_lg==2.2.0) (0.15.2)
Requirement already satisfied: python-dateutil<3.0.0,>=2.1 in /opt/conda/lib/python3.6/site-packages (from botocore<1.16.0,>=1.15.32->boto3->transformers<2.1.0,>=2.0.0->spacy-transformers>=0.5.0-  >en_trf_bertbaseuncased_lg==2.2.0) (2.8.1)
Building wheels for collected packages: en-trf-bertbaseuncased-lg, spacy-transformers, torchcontrib  
  Building wheel for en-trf-bertbaseuncased-lg (setup.py) ... done  
  Created wheel for en-trf-bertbaseuncased-lg: filename=en_trf_bertbaseuncased_lg-2.2.0-py3-none-any.whl size=405819943 sha256=b5c1d91dbcea394dfc3a77f5c54184e6194af2868652bcb06d2e87970a65af6d  
  Stored in directory: /tmp/pip-ephem-wheel-cache-_iaz4fe9/wheels/ec/ee/57/ab87a942c552daca7cf69047b2180d8b71f2917b584e37d2d1  
  Building wheel for spacy-transformers (setup.py) ... done  
  Created wheel for spacy-transformers: filename=spacy_transformers-0.5.1-py2.py3-none-any.whl size=52835 sha256=02c3b327ee3828b6e83913e155d64057392a37382edef8b7a8af023b12b7b116  
  Stored in directory: /tmp/pip-ephem-wheel-cache-_iaz4fe9/wheels/5b/fc/ec/3f07ac701437c5b06685db7fc64d9a963d1e6c8fcff62922f3  
  Building wheel for torchcontrib (setup.py) ... done  
  Created wheel for torchcontrib: filename=torchcontrib-0.0.2-py3-none-any.whl size=7532 sha256=13b106f410c0d633a765798bc1a2aa8c9f4ab441d57c0a5634d04cada0bbc765  
  Stored in directory: /tmp/pip-ephem-wheel-cache-_iaz4fe9/wheels/cc/fc/93/b060f81a70c264ecc80a21158efc6034712bd6144f9fe53d02  
Successfully built en-trf-bertbaseuncased-lg spacy-transformers torchcontrib  
Installing collected packages: transformers, torchcontrib, dataclasses, spacy-transformers, en-trf-bertbaseuncased-lg  
  Attempting uninstall: transformers  
    Found existing installation: transformers 2.7.0  
    Uninstalling transformers-2.7.0:  
      Successfully uninstalled transformers-2.7.0  
  Attempting uninstall: dataclasses  
    Found existing installation: dataclasses 0.7  
    Uninstalling dataclasses-0.7:  
      Successfully uninstalled dataclasses-0.7  
Successfully installed dataclasses-0.6 en-trf-bertbaseuncased-lg-2.2.0 spacy-transformers-0.5.1 torchcontrib-0.0.2 transformers-2.0.0  
✔ Download and installation successful  
You can now load the model via spacy.load('en_trf_bertbaseuncased_lg')  
True  
done  
CPU times: user 8.87 s, sys: 2.67 s, total: 11.5 s  
Wall time: 2min 16s  

In [ ]:
# define and train model with custom parameters
nlp = spacy.load(r"/opt/conda/lib/python3.6/site-packages/en_trf_bertbaseuncased_lg/en_trf_bertbaseuncased_lg-2.2.0")

In [ ]:
os.mkdir("../working/epoch_0")
os.mkdir("../working/epoch_1")
os.mkdir("../working/epoch_2")
os.mkdir("../working/epoch_3")
os.mkdir("../working/epoch_4")
os.mkdir("../working/epoch_5")
os.mkdir("../working/epoch_6")

In [ ]:
punctuation = '#$%&\'()*+,-:;<=>@[]^_`{|}~' #removed !.?"\/
punctl = [p for p in punctuation]
emos = [":)", ":(",":-)",":-("]

def preprocessHTML(raw):
    tex = str(raw).lower()
    cleanr = re.compile('<.*?>') # remove html tags
    tex = re.sub(cleanr, '', tex)
    tex = re.sub('http[s]?://\S+', '', tex)
    tex = re.sub(r'([)(])\1+', r'\1', tex).replace("/\s{2,}/g"," ") # if it finds ) or ( repeated more than once, it just substitute with one ex )) --> )
    tex = " ".join([word.translate(str.maketrans({key: "" for key in punctl}))  if word not in emos else word for word in tex.split(" ")]) # removes punctuation but keeps smiles
    tex = re.sub(r'([!.?"])\1+', r'\1 \1 \1', tex).replace("/\s{2,}/g"," ") # substitute all repeated ! . ? and "  with 3 instances es ??? ---> ? ? ? (and similar for . and !) #^\w\s
    tex = tex.translate(str.maketrans({key: " {0} ".format(key) for key in ["!","?",".",'"',"/","\\"]})).translate(str.maketrans('','','1234567890')) # puts spaces around punctuation and removes numbers
    tex = re.sub(r'[\\/]',"",tex)
    tex = " ".join(tex.strip().split())
    tex = tex.strip()
    return tex

In [ ]:
%%time
docneg = []
docpos = []
labpos = []
labneg = []
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename,encoding="utf8")
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

def process_docs(directory,doc,labels):
    l = len(listdir(directory))
    pbar = ProgressBar(widgets=[Percentage(), Bar()], maxval=l).start()
    i = 0
    # walk through all files in the folder
    for filename in listdir(directory):
        # skip files that do not have the right extension
        if not filename.endswith(".txt"):
            continue
        # create the full path of the file to open
        path = directory + '/' + filename
        # load document
        doc.append(preprocessHTML(load_doc(path)))
        labels.append(filename[-5:-4])
        i += 1
        pbar.update(i)
    pbar.finish()


directoryn = r'../input/bertdata/aclImdb/train/neg'
directoryp = r'../input/bertdata/aclImdb/train/pos'
process_docs(directoryn,docneg,labneg)
process_docs(directoryp,docpos,labpos)
labpos = [int(i) for i in labpos]
labneg = [int(i) for i in labneg]
labposb = [1 for i in labpos]
labnegb = [0 for i in labneg]
print("len(docpos) = ", len(docpos), "len(docneg) = ", len(docneg))
print("len(labpos) = ", len(labpos), "len(labneg) = ", len(labneg))
if len(docpos) == len(labpos) and len(docneg) == len(labneg):
    print("safe to proceed")
else:
    print("WARNING: LABELS ARE NOT IN THE SAME NUMBER AS DATA, TERMINATION SUGGESTED")


train_texts = docpos[:10000] + docneg[:10000]
train_labels = labposb[:10000] + labnegb[:10000]
train_labels = [{'cats': {'1': label == 1,'0': label == 0}} for label in train_labels]
print("len(train_labels) = ", len(train_labels), "len(train_texts) = ", len(train_texts), len(train_texts) == len(train_labels))



val_texts = docpos[10000:] + docneg[10000:]
val_labels = labposb[10000:] + labnegb[10000:]
print("len(val_labels) = ", len(val_labels), "len(val_texts) = ", len(val_texts), len(val_texts) == len(val_labels))

val_texts, val_labels = shuffle(val_texts, val_labels , random_state=0)

train_data  = list(zip(train_texts, train_labels))

100%|#########################################################################|  
100%|#########################################################################|  
len(docpos) =  12500 len(docneg) =  12500  
len(labpos) =  12500 len(labneg) =  12500  
safe to proceed  
len(train_labels) =  20000 len(train_texts) =  20000 True  
len(val_labels) =  5000 len(val_texts) =  5000 True  
CPU times: user 48.6 s, sys: 3.01 s, total: 51.6 s  
Wall time: 1min 25s  

In [ ]:
# evaluate roc
def evaluate_roc(nlp):
    docs = []
    pbar = ProgressBar(widgets=[Percentage(), Bar()], maxval = len(val_texts)+1).start()  #8387 is the total number of iterations needed for an unbalanced 0.8 trainset. 0.04 of unbalanced would be approx 1540 . 4595 for lemmatized balanced. 8420 for lemmatized unbalanced
    i = 0
    f = 0
    for tex in val_texts:
        try:
            docs.append(nlp(tex)) 
            i = i+1
        except:
            print("Error occured at val_text number: ", str(i))
            break
        pbar.update(i)
    pbar.finish()
    y_pred = [doc.cats["1"] for doc in docs]
    print("succeded up to: ", i)
    roc = roc_auc_score(val_labels[:i], y_pred)
    return roc

In [ ]:
%%time


print(nlp.pipe_names) # ["sentencizer", "trf_wordpiecer", "trf_tok2vec"]
# Create the TextCategorizer with exclusive classes and "bow" (bag of words) architecture.
if 'trf_textcat' not in nlp.pipe_names:
    textcat = nlp.create_pipe("trf_textcat", config={"exclusive_classes": True , "architecture": "softmax_last_hidden", "words_per_batch": 0}) 
    textcat.add_label("1")
    textcat.add_label("0")
    nlp.add_pipe(textcat, last = True)
else:
    print("something went wrong")
    textcat = nlp.get_pipe('trf_textcat')
#nlp.add_pipe(nlp.create_pipe("sentencizer"))



random.seed(1)
spacy.util.fix_random_seed(1)

nlp.vocab.vectors.name = 'spacy_pretrained_vectors'


print("random seeds set")

losses  = {}
rocs = []
run_title = "bertbulg_80_softmax_wpb0"
output = ""

print("strings and lists initialized")


#learning process
pipe_exceptions = ['trf_textcat', "trf_wordpiecer","trf_tok2vec"] #("trf_wordpiecer", "trf_tok2vec) are required, #https://github.com/explosion/spaCy/blob/master/examples/training/train_textcat.py
print("pipe_exceptions defined")
print(nlp.pipe_names)
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
print("other_pipes defined")
with nlp.disable_pipes(*other_pipes):  # only train textcat
    optimizer = nlp.resume_training()
    optimizer.alpha = 0.001
    optimizer.trf_weight_decay = 0.005
    optimizer.L2 = 0.0
    learn_rate=2e-5
    batch_size  = 8
    learn_rates = cyclic_triangular_rate(learn_rate / 3, learn_rate * 3, 2 * len(train_data) // batch_size)
    for epoch in range(7):
        random.shuffle(train_data)
        print("data shuffled")
        # Create the batch generator with batch size = 8
        batches = minibatch(train_data, size=batch_size)
        print("batches created")
        optimizer.trf_lr = next(learn_rates)
        # Iterate through minibatches
        pbar = ProgressBar(widgets=[Percentage(), Bar()], maxval=2525).start()  
        i = 0                                                                    
        for batch in batches:
            # Each batch is a list of (text, label) but we need to
            # send separate lists for texts and labels to update().
            # This is a quick way to split a list of tuples into lists
            texts1, labels = zip(*batch)
            nlp.update(texts1, labels, sgd = optimizer, losses=losses, drop = 0.1)
            i += 1
            pbar.update(i)
        pbar.finish()
        print("i = ",  i)
        with nlp.use_params(optimizer.averages):   
            rocs.append(evaluate_roc(nlp))
            nlp.to_disk(r"../working/"+"epoch_"+str(epoch)+"/bertbulg_80_softmax_wpb0_"+str(epoch))
            output += f"    epoch = {epoch}, roc = {rocs[-1]} , loss = {losses} \n "  
            print( "epoch = ",epoch, "roc = " , rocs[-1] ,"loss =",losses  ,"i = ", i)  



with open(r"../working/" + run_title + ".txt", "a") as file: #name
    file.write(run_title +"\n" + output)
    file.close()

['sentencizer', 'trf_wordpiecer', 'trf_tok2vec'] <br>
pheww<br>
  0%|                                                                         |<br>
random seeds set<br>
strings and lists initialized<br>
pipe_exceptions defined<br>
['sentencizer', 'trf_wordpiecer', 'trf_tok2vec', 'trf_textcat']<br>
other_pipes defined<br>
data shuffled<br>
batches created<br>
100%|#########################################################################|<br>
  0%|                                                                         |<br>
i =  2500<br>
evaluate1<br>
100%|#########################################################################|<br>
Error occured at val_text number:  3496<br>
evaluate2<br>
succeded up to:  3496<br>
evaluate3<br>
evaluate4<br>
<br>
  0%|                                                                         |<br>
epoch =  0 roc =  0.97479066403316 loss = {'trf_textcat': 6.877855688240999} i =  2500<br>
data shuffled<br>
batches created<br>
100%|#########################################################################|<br>
  0%|                                                                         |<br>
i =  2500  
evaluate1  
100%|#########################################################################|   
Error occured at val_text number:  3795 
evaluate2  
succeded up to:  3795  
evaluate3  
evaluate4  
  
  0%|                                                                         |  
epoch =  1 roc =  0.9755197732706444 loss = {'trf_textcat': 10.744955211586598} i =  2500  
data shuffled  
batches created  
100%|#########################################################################|  
  0%|                                                                         |  
i =  2500  
evaluate1  
100%|#########################################################################|  
Error occured at val_text number:  3878  
evaluate2  
succeded up to:  3878  
evaluate3  
evaluate4  
  
  0%|                                                                         |  
epoch =  2 roc =  0.9751145844637511 loss = {'trf_textcat': 12.884738852677117} i =  2500  